## 1. GAN기본모델 구현하기

In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./mnist/data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [17]:
total_epoch=100
batch_size=100
learning_rate=0.0002
n_hidden=256
n_input=28*28
n_noise=128 # G입력으로 사용할 노이즈 크기

- GAN은 비지도학습
- 따라서 오토인코더 처럼 Y를 사용하지 않는다ㅣ.

In [18]:
X=tf.placeholder(tf.float32,[None,n_input])
Z=tf.placeholder(tf.float32,[None,n_noise]) # 노이즈를 입력할 Z

In [19]:
G_W1=tf.Variable(tf.random_normal([n_noise,n_hidden],stddev=0.01))
G_b1=tf.Variable(tf.zeros([n_hidden]))
G_W2=tf.Variable(tf.random_normal([n_hidden,n_input],stddev=0.01))
G_b2=tf.Variable(tf.zeros([n_input])) # 출력이 img size > 28*28=784

In [20]:
D_W1=tf.Variable(tf.random_normal([n_input,n_hidden],stddev=0.01))
D_b1=tf.Variable(tf.zeros([n_hidden]))
D_W2=tf.Variable(tf.random_normal([n_hidden,1],stddev=0.01))
D_b2=tf.Variable(tf.zeros([1])) # 이진출력 0 또는 1

In [21]:
def generator(noise_z):
    hidden=tf.nn.relu(tf.matmul(noise_z,G_W1)+G_b1)
    output=tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output

In [22]:
def discriminator(inputs):
    hidden=tf.nn.relu(tf.matmul(inputs,D_W1)+D_b1)
    output=tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return output

In [33]:
def get_noise(batch_size,n_noise):
    return np.random.normal(size=(batch_size,n_noise))

In [24]:
G=generator(Z) # fake 생성
D_gene=discriminator(G) # fake
D_real=discriminator(X) # real

In [25]:
loss_D=tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))
loss_G=tf.reduce_mean(tf.log(D_gene))

- GAN의 학습은 loss_D, loss_G 모두를 최대화하는 것
- loss_D와 loss_G는 서로 연관되어있어서 D가 높아지면 G는 낮아지거나 G가 높아지면 D가 낮아질 것
- **G 학습시 D의 동결이 중요하다**

In [26]:
D_var_list=[D_W1,D_b1,D_W2,D_b2]
G_var_list=[G_W1,G_b1,G_W2,G_b2]

- GAN논문에 따르면 loss를 최대화 해야하나 최적화에 쓸 수 있는 함수는 minimize뿐
- 따라서 최적화 하려는 loss_D와 loss_G에 음수부호룰 붙여준다.

In [27]:
train_D=tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list=D_var_list)
train_G=tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list)

In [31]:
# 학습 시작
sess=tf.Session()
sess.run(tf.global_variables_initializer())
total_batch=int(mnist.train.num_examples/batch_size)
loss_val_D=0.0
loss_val_G=0.0

In [35]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        noise=get_noise(batch_size,n_noise)
        _,loss_val_D=sess.run([train_D,loss_D],feed_dict={X:batch_xs,Z:noise})
        _,loss_val_G=sess.run([train_G,loss_G],feed_dict={Z:noise})
        print('epoch:','%04d' %epoch, 'D loss: {:.4}'.format(loss_val_D),'G loss: {:.4}'.format(loss_val_G))
    if epoch==0 or (epoch+1)%10 ==0:
        sample_size=10
        noise=get_noise(sample_size,n_noise)
        samples=sess.run(G,feed_dict={Z:noise})
        
        fig,ax=plt.subplots(1,sample_size,figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off() # 축을 끄는 것.
            ax[i].imshow(np.reshape(samples[i],(28,28))) # imshow > data를 2D의 이미지로 변환, 28*28 size로 reshape 후 시각화
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
        plt.close(fig)
print("최적화 완료")

epoch: 0000 D loss: -0.4346 G loss: -2.354
epoch: 0000 D loss: -0.6 G loss: -2.285
epoch: 0000 D loss: -0.495 G loss: -2.016
epoch: 0000 D loss: -0.5049 G loss: -2.166
epoch: 0000 D loss: -0.4988 G loss: -2.168
epoch: 0000 D loss: -0.4348 G loss: -2.075
epoch: 0000 D loss: -0.4623 G loss: -2.022
epoch: 0000 D loss: -0.5818 G loss: -2.002
epoch: 0000 D loss: -0.6006 G loss: -2.095
epoch: 0000 D loss: -0.5698 G loss: -1.963
epoch: 0000 D loss: -0.5712 G loss: -1.943
epoch: 0000 D loss: -0.5097 G loss: -1.93
epoch: 0000 D loss: -0.6447 G loss: -1.984
epoch: 0000 D loss: -0.5052 G loss: -2.101
epoch: 0000 D loss: -0.5808 G loss: -2.179
epoch: 0000 D loss: -0.5553 G loss: -2.155
epoch: 0000 D loss: -0.5758 G loss: -2.25
epoch: 0000 D loss: -0.5779 G loss: -2.123
epoch: 0000 D loss: -0.6648 G loss: -2.021
epoch: 0000 D loss: -0.4561 G loss: -2.018
epoch: 0000 D loss: -0.5508 G loss: -1.882
epoch: 0000 D loss: -0.4845 G loss: -2.004
epoch: 0000 D loss: -0.4729 G loss: -1.924
epoch: 0000 D los

epoch: 0000 D loss: -0.6465 G loss: -2.014
epoch: 0000 D loss: -0.5023 G loss: -2.173
epoch: 0000 D loss: -0.4774 G loss: -2.034
epoch: 0000 D loss: -0.5701 G loss: -1.942
epoch: 0000 D loss: -0.5914 G loss: -2.081
epoch: 0000 D loss: -0.5531 G loss: -2.076
epoch: 0000 D loss: -0.5353 G loss: -2.09
epoch: 0000 D loss: -0.4656 G loss: -2.163
epoch: 0000 D loss: -0.5627 G loss: -2.02
epoch: 0000 D loss: -0.5597 G loss: -2.003
epoch: 0000 D loss: -0.4972 G loss: -2.153
epoch: 0000 D loss: -0.5049 G loss: -2.323
epoch: 0000 D loss: -0.5839 G loss: -2.231
epoch: 0000 D loss: -0.5772 G loss: -2.357
epoch: 0000 D loss: -0.6005 G loss: -2.119
epoch: 0000 D loss: -0.5601 G loss: -2.143
epoch: 0000 D loss: -0.5884 G loss: -2.116
epoch: 0000 D loss: -0.4954 G loss: -2.206
epoch: 0000 D loss: -0.6098 G loss: -2.0
epoch: 0000 D loss: -0.6845 G loss: -2.045
epoch: 0000 D loss: -0.5915 G loss: -1.937
epoch: 0000 D loss: -0.5204 G loss: -2.064
epoch: 0000 D loss: -0.6461 G loss: -1.895
epoch: 0000 D l

epoch: 0000 D loss: -0.4774 G loss: -2.084
epoch: 0000 D loss: -0.5698 G loss: -1.992
epoch: 0000 D loss: -0.4685 G loss: -2.273
epoch: 0000 D loss: -0.6134 G loss: -2.132
epoch: 0000 D loss: -0.5417 G loss: -2.171
epoch: 0000 D loss: -0.475 G loss: -2.287
epoch: 0000 D loss: -0.5513 G loss: -2.181
epoch: 0000 D loss: -0.479 G loss: -2.203
epoch: 0000 D loss: -0.5212 G loss: -2.188
epoch: 0000 D loss: -0.508 G loss: -2.206
epoch: 0000 D loss: -0.4657 G loss: -2.295
epoch: 0000 D loss: -0.5047 G loss: -2.311
epoch: 0000 D loss: -0.5228 G loss: -2.129
epoch: 0000 D loss: -0.562 G loss: -2.098
epoch: 0000 D loss: -0.5963 G loss: -1.992
epoch: 0000 D loss: -0.6141 G loss: -2.045
epoch: 0000 D loss: -0.4495 G loss: -2.098
epoch: 0000 D loss: -0.4881 G loss: -2.182
epoch: 0000 D loss: -0.4925 G loss: -2.219
epoch: 0000 D loss: -0.5386 G loss: -1.965
epoch: 0000 D loss: -0.5196 G loss: -2.248
epoch: 0000 D loss: -0.5466 G loss: -2.08
epoch: 0000 D loss: -0.477 G loss: -2.14
epoch: 0000 D loss

epoch: 0001 D loss: -0.4688 G loss: -2.145
epoch: 0001 D loss: -0.4297 G loss: -2.031
epoch: 0001 D loss: -0.4262 G loss: -2.194
epoch: 0001 D loss: -0.4685 G loss: -1.99
epoch: 0001 D loss: -0.506 G loss: -2.118
epoch: 0001 D loss: -0.4907 G loss: -2.249
epoch: 0001 D loss: -0.436 G loss: -2.175
epoch: 0001 D loss: -0.5247 G loss: -2.256
epoch: 0001 D loss: -0.4733 G loss: -2.283
epoch: 0001 D loss: -0.4034 G loss: -2.326
epoch: 0001 D loss: -0.4007 G loss: -2.305
epoch: 0001 D loss: -0.6287 G loss: -2.257
epoch: 0001 D loss: -0.4763 G loss: -2.362
epoch: 0001 D loss: -0.5013 G loss: -2.203
epoch: 0001 D loss: -0.5142 G loss: -2.211
epoch: 0001 D loss: -0.4978 G loss: -2.215
epoch: 0001 D loss: -0.4398 G loss: -2.179
epoch: 0001 D loss: -0.454 G loss: -2.175
epoch: 0001 D loss: -0.4701 G loss: -2.154
epoch: 0001 D loss: -0.5156 G loss: -2.087
epoch: 0001 D loss: -0.4011 G loss: -2.186
epoch: 0001 D loss: -0.533 G loss: -2.26
epoch: 0001 D loss: -0.5025 G loss: -2.259
epoch: 0001 D los

epoch: 0001 D loss: -0.3231 G loss: -2.543
epoch: 0001 D loss: -0.3732 G loss: -2.632
epoch: 0001 D loss: -0.466 G loss: -2.52
epoch: 0001 D loss: -0.3972 G loss: -2.475
epoch: 0001 D loss: -0.3893 G loss: -2.679
epoch: 0001 D loss: -0.4288 G loss: -2.375
epoch: 0001 D loss: -0.4322 G loss: -2.482
epoch: 0001 D loss: -0.3551 G loss: -2.443
epoch: 0001 D loss: -0.4564 G loss: -2.398
epoch: 0001 D loss: -0.3821 G loss: -2.438
epoch: 0001 D loss: -0.4461 G loss: -2.457
epoch: 0001 D loss: -0.367 G loss: -2.338
epoch: 0001 D loss: -0.3534 G loss: -2.445
epoch: 0001 D loss: -0.3797 G loss: -2.475
epoch: 0001 D loss: -0.4018 G loss: -2.523
epoch: 0001 D loss: -0.4882 G loss: -2.288
epoch: 0001 D loss: -0.4246 G loss: -2.258
epoch: 0001 D loss: -0.4144 G loss: -2.356
epoch: 0001 D loss: -0.3471 G loss: -2.397
epoch: 0001 D loss: -0.4443 G loss: -2.457
epoch: 0001 D loss: -0.3984 G loss: -2.57
epoch: 0001 D loss: -0.526 G loss: -2.412
epoch: 0001 D loss: -0.4391 G loss: -2.313
epoch: 0001 D lo

...

epoch: 0098 D loss: -0.6446 G loss: -2.156
epoch: 0098 D loss: -0.6403 G loss: -2.354
epoch: 0098 D loss: -0.5289 G loss: -2.369
epoch: 0098 D loss: -0.6145 G loss: -2.181
epoch: 0098 D loss: -0.6267 G loss: -1.98
epoch: 0098 D loss: -0.5915 G loss: -1.976
epoch: 0098 D loss: -0.6004 G loss: -1.925
epoch: 0098 D loss: -0.5785 G loss: -1.9
epoch: 0098 D loss: -0.5255 G loss: -2.03
epoch: 0098 D loss: -0.601 G loss: -2.28
epoch: 0098 D loss: -0.6033 G loss: -2.154
epoch: 0098 D loss: -0.6237 G loss: -2.319
epoch: 0098 D loss: -0.5623 G loss: -2.565
epoch: 0098 D loss: -0.5824 G loss: -2.614
epoch: 0098 D loss: -0.7034 G loss: -2.387
epoch: 0098 D loss: -0.5944 G loss: -2.294
epoch: 0098 D loss: -0.616 G loss: -2.148
epoch: 0098 D loss: -0.4909 G loss: -2.248
epoch: 0098 D loss: -0.7195 G loss: -2.086
epoch: 0098 D loss: -0.6553 G loss: -1.944
epoch: 0098 D loss: -0.7065 G loss: -2.171
epoch: 0098 D loss: -0.5408 G loss: -2.372
epoch: 0098 D loss: -0.5633 G loss: -2.291
epoch: 0098 D loss

epoch: 0098 D loss: -0.6509 G loss: -2.385
epoch: 0098 D loss: -0.5608 G loss: -2.428
epoch: 0098 D loss: -0.5873 G loss: -2.394
epoch: 0098 D loss: -0.6427 G loss: -2.318
epoch: 0098 D loss: -0.607 G loss: -2.186
epoch: 0098 D loss: -0.6591 G loss: -2.128
epoch: 0098 D loss: -0.5179 G loss: -2.254
epoch: 0098 D loss: -0.7165 G loss: -2.129
epoch: 0098 D loss: -0.7064 G loss: -1.915
epoch: 0098 D loss: -0.7057 G loss: -1.748
epoch: 0098 D loss: -0.5566 G loss: -2.085
epoch: 0098 D loss: -0.6932 G loss: -1.757
epoch: 0098 D loss: -0.6557 G loss: -2.123
epoch: 0098 D loss: -0.6145 G loss: -2.085
epoch: 0098 D loss: -0.593 G loss: -2.264
epoch: 0098 D loss: -0.6577 G loss: -2.305
epoch: 0098 D loss: -0.5466 G loss: -2.376
epoch: 0098 D loss: -0.6193 G loss: -2.048
epoch: 0098 D loss: -0.5357 G loss: -2.446
epoch: 0098 D loss: -0.4895 G loss: -2.46
epoch: 0098 D loss: -0.597 G loss: -2.484
epoch: 0098 D loss: -0.5929 G loss: -2.365
epoch: 0098 D loss: -0.6392 G loss: -2.473
epoch: 0098 D l

epoch: 0099 D loss: -0.5558 G loss: -2.379
epoch: 0099 D loss: -0.4897 G loss: -2.334
epoch: 0099 D loss: -0.5077 G loss: -2.378
epoch: 0099 D loss: -0.5646 G loss: -2.259
epoch: 0099 D loss: -0.484 G loss: -2.216
epoch: 0099 D loss: -0.6188 G loss: -2.19
epoch: 0099 D loss: -0.6001 G loss: -2.227
epoch: 0099 D loss: -0.493 G loss: -2.497
epoch: 0099 D loss: -0.5531 G loss: -2.325
epoch: 0099 D loss: -0.5935 G loss: -2.54
epoch: 0099 D loss: -0.4659 G loss: -2.603
epoch: 0099 D loss: -0.5626 G loss: -2.44
epoch: 0099 D loss: -0.5548 G loss: -2.169
epoch: 0099 D loss: -0.661 G loss: -2.186
epoch: 0099 D loss: -0.5119 G loss: -2.397
epoch: 0099 D loss: -0.6537 G loss: -2.189
epoch: 0099 D loss: -0.6024 G loss: -2.288
epoch: 0099 D loss: -0.5183 G loss: -2.434
epoch: 0099 D loss: -0.6874 G loss: -1.945
epoch: 0099 D loss: -0.5545 G loss: -2.21
epoch: 0099 D loss: -0.6244 G loss: -1.923
epoch: 0099 D loss: -0.5082 G loss: -2.247
epoch: 0099 D loss: -0.6606 G loss: -2.168
epoch: 0099 D loss

epoch: 0099 D loss: -0.5847 G loss: -2.128
epoch: 0099 D loss: -0.602 G loss: -2.254
epoch: 0099 D loss: -0.5688 G loss: -2.186
epoch: 0099 D loss: -0.6408 G loss: -1.999
epoch: 0099 D loss: -0.638 G loss: -2.161
epoch: 0099 D loss: -0.4893 G loss: -2.111
epoch: 0099 D loss: -0.601 G loss: -2.362
epoch: 0099 D loss: -0.6822 G loss: -2.434
epoch: 0099 D loss: -0.6335 G loss: -2.082
epoch: 0099 D loss: -0.6152 G loss: -2.274
epoch: 0099 D loss: -0.481 G loss: -2.541
epoch: 0099 D loss: -0.5833 G loss: -2.148
epoch: 0099 D loss: -0.6101 G loss: -2.177
epoch: 0099 D loss: -0.6011 G loss: -1.933
epoch: 0099 D loss: -0.5863 G loss: -2.23
epoch: 0099 D loss: -0.5801 G loss: -2.062
epoch: 0099 D loss: -0.5642 G loss: -2.297
epoch: 0099 D loss: -0.5584 G loss: -2.113
epoch: 0099 D loss: -0.7649 G loss: -2.181
epoch: 0099 D loss: -0.6352 G loss: -2.193
epoch: 0099 D loss: -0.524 G loss: -2.372
epoch: 0099 D loss: -0.6292 G loss: -2.398
epoch: 0099 D loss: -0.6307 G loss: -2.519
epoch: 0099 D los

최적화 완료


## 2. 원하는 숫자 생성하기
- 숫자를 무작위로 생성하지 않고 원하는 숫자를 지정해 생성하는 모델 만들기
- 노이즈에 레이블데이터를 힌트로 넣어주는 방법 사용
- Y값 추가

In [36]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./mnist/data/",one_hot=True)

total_epoch=100
batch_size=100
learning_rate=0.0002
n_hidden=256
n_input=28*28
n_noise=128 # G입력으로 사용할 노이즈 크기
n_class=10

X=tf.placeholder(tf.float32,[None,n_input])
Y=tf.placeholder(tf.float32,[None,n_class])
Z=tf.placeholder(tf.float32,[None,n_noise]) # 노이즈를 입력할 Z

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


- 여기서는 변수들을 선언하지 않고 tf.layers를 사용해보기
    - 변수를 선언하지 않고 tf.variables_scope 를 이용해 스코프를 지정할 수 있다. (https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/variable_scope/)
- 그런 뒤 tf.concat함수를 이용해 noise값에 labels정보를 간단하게 추가. 

In [37]:
def generator(noise_z,labels):
    with tf.variable_scope('generator'):
        input=tf.concat([noise_z,labels],1)
        hidden=tf.layers.dense(input,n_hidden,activation=tf.nn.relu) #이전에는 활성화함수(X*W +b) 형태
        output=tf.layers.dense(hidden,n_input,activation=tf.nn.sigmoid)
    return output

In [39]:
# 진짜이미지와 가짜이미지를 판별할때 똑같은 변수를 사용해야 한다. 따라서 scope.reuse_variables를 이용해 이전에 사용한 변수를 재사용 하도록 한다.
def discriminator(inputs,labels,reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
            input=tf.concat([input,labels],1)
            hidden=tf.layers.dense(input,n_hidden,activation=tf.nn.relu)
            output=tf.layers.dense(n_hidden,1,activation=None) # 활성화함수를 사용하지 않은 이유는 손실값 계산에 sigmoid cross entropy with logits를 사용하기 위함
    return output